In [18]:
import mnist_loader
import numpy as np
import random


In [65]:
td, vd, tesd = mnist_loader.load_data_wrapper()

print(td[0])


(array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 

In [21]:
sizes = (784, 30, 10)


num_layers = len(sizes)
#y * 1
biases = [np.random.randn(y, 1) for y in sizes[1:]]\
#return y * x matrix, first is 5 * 20
weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]






In [22]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

#导数
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [47]:
def feedforward(a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(biases, weights):
            
            a = sigmoid(np.dot(w, a)+b)
            
        return a

In [24]:
#x = np.ones((10, 1))
x = td[0][0]
b = feedforward(x)
#print(b)




30
10


In [25]:
 def cost_derivative(output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations - y)

In [42]:
 def backprop(x, y):
        global biases, weights
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros_like(b) for b in biases]
        nabla_w = [np.zeros_like(w) for w in weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        
        #store the z and weighted z(it is a)
        for b, w in zip(biases, weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)


In [43]:
 def update_mini_batch(mini_batch, eta):
        global biases, weights
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in biases]
        nabla_w = [np.zeros(w.shape) for w in weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, nabla_w)]
        biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, nabla_b)]

In [44]:
 def evaluate(test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)


In [69]:
def evaluate1(test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

In [76]:
 def SGD(training_data, epochs, mini_batch_size, eta, test_data=None):
        global biases, weights
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        print("Test data size if {0}".format(n))
        for j in range(epochs):
            #this ???not effective
            random.shuffle(training_data)
            mini_batches = [training_data[k : k +  mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: Study Acc:{1:2f}, Test Acc: {2} / {3}".format(j, evaluate1(training_data)/n, evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

In [81]:
SGD(td[0:1000], 30, 10, 3.0, test_data=tesd)

Test data size if 1000
Epoch 0: Study Acc:0.991000, Test Acc: 9485 / 10000
Epoch 1: Study Acc:0.991000, Test Acc: 9485 / 10000
Epoch 2: Study Acc:0.991000, Test Acc: 9486 / 10000
Epoch 3: Study Acc:0.991000, Test Acc: 9484 / 10000
Epoch 4: Study Acc:0.991000, Test Acc: 9480 / 10000
Epoch 5: Study Acc:0.991000, Test Acc: 9482 / 10000
Epoch 6: Study Acc:0.991000, Test Acc: 9484 / 10000
Epoch 7: Study Acc:0.991000, Test Acc: 9485 / 10000
Epoch 8: Study Acc:0.991000, Test Acc: 9485 / 10000
Epoch 9: Study Acc:0.991000, Test Acc: 9484 / 10000
Epoch 10: Study Acc:0.991000, Test Acc: 9485 / 10000
Epoch 11: Study Acc:0.991000, Test Acc: 9486 / 10000
Epoch 12: Study Acc:0.991000, Test Acc: 9486 / 10000
Epoch 13: Study Acc:0.991000, Test Acc: 9485 / 10000
Epoch 14: Study Acc:0.992000, Test Acc: 9483 / 10000
Epoch 15: Study Acc:0.991000, Test Acc: 9483 / 10000
Epoch 16: Study Acc:0.992000, Test Acc: 9483 / 10000
Epoch 17: Study Acc:0.991000, Test Acc: 9480 / 10000
Epoch 18: Study Acc:0.992000, Tes